## Load Libraries and Firmware

In [8]:
USE_LOCAL_QICK = True

if USE_LOCAL_QICK:
    import sys
    import os
    print(sys.path.insert(0,"/home/dmartin2/Projects/qick_internal_b_43_testbench/qick_lib/"))
    print(sys.path)


# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qick import *

# for now, all the tProc v2 classes need to be individually imported (can't use qick.*)

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

print(get_version())

None
['/home/dmartin2/Projects/qick_internal_b_43_testbench/qick_lib/', '/home/dmartin2/Projects/qick_internal_b_43_testbench/qick_lib/', '/home/dmartin2/Projects/qick_internal_b_43_testbench/qick_lib/', '/usr/lib64/python39.zip', '/usr/lib64/python3.9', '/usr/lib64/python3.9/lib-dynload', '', '/home/dmartin2/venv/lib64/python3.9/site-packages', '/home/dmartin2/venv/lib/python3.9/site-packages']
0.2.367


In [9]:
QICK_EMU = True
# soccfg_file_name = '/home/xilinx/jupyter_notebooks/qick/firmware/testbench/qick_testbench/soccfg.json'
soccfg_file_name = '../../firmware/testbench/qick_testbench/soccfg.json'

if QICK_EMU is False:
    # soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/2024-09-28_216_tprocv2r21_rfb_standard/qick_216_rfb.bit')
    # soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/2025-06-15_216_tprocv2r24_standard/qick_216.bit')
    # soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_216_standard_1ch_250822_1/qick_216.bit')
    # soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_216_standard_1ch_250828_1/qick_216.bit')
    # soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_216_standard_1ch_250828_2/qick_216.bit')
    # soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_216_standard_1ch_250829_1/qick_216.bit')
    soc = QickSoc('/home/xilinx/jupyter_notebooks/fw/qick_tprocv2_216_standard_1ch_250829_2/qick_216.bit')
    soccfg = soc

    # Dump Board SoC configuration in json format and create file
    soccfg_json = soc.dump_cfg(gen_file_path=soccfg_file_name)
    # print(soccfg_json)
else:
    soc = None
    # Load soccfg.json file as a new socemucfg object
    soccfg = QickConfig(cfg=soccfg_file_name)

print(soccfg)

QICK running on ZCU216, software version 0.2.367

Firmware configuration (built Fri Aug 29 12:51:57 2025):

	Global clocks (MHz): tProc dispatcher timing 430.080, RF reference 245.760
	Groups of related clocks: [tProc timing clock, DAC tile 1, DAC tile 2, DAC tile 3], [DAC tile 0], [ADC tile 2]

	1 signal generator channels:
	0:	axis_signal_gen_v6 - fs=9584.640 Msps, fabric=599.040 MHz
		envelope memory: 65536 complex samples (6.838 us)
		32-bit DDS, range=9584.640 MHz
		DAC tile 0, blk 0 is 0_228 on JHC1, or QICK box DAC port 0

	1 readout channels:
	0:	axis_dyn_readout_v1 - configured by tProc output 4
		fs=2457.600 Msps, decimated=307.200 MHz, 32-bit DDS, range=2457.600 MHz
		axis_avg_buffer v1.2 (has edge counter, no weights)
		memory 8192 accumulated, 4096 decimated (13.333 us)
		triggered by tport 10, pin 0, feedback to tProc input 0
		ADC tile 2, blk 0 is 0_226 on JHC7, or QICK box ADC port 4

	8 digital output pins:
	0:	PMOD0_0_LS
	1:	PMOD0_1_LS
	2:	PMOD0_2_LS
	3:	PMOD0_3_LS
	4

## basic multi-pulse program
Just a "reps" loop, no sweeping - like v1 AveragerProgram.

In v2, you define all of your pulses, and then you can play them as needed. Contrast to v1, where only one pulse could be defined on a generator at a time.

In [10]:
GEN_CH = 0
RO_CH = 0
TRIG_TIME = 0.40
FREQ = 100
# FREQ = 0

class MultiPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        ramp_len = 0.2
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myflattop", ro_ch=ro_ch, 
                       style="flat_top", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       length=0.1,
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mygaus", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=0.2, 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0,
                      )

        self.add_pulse(ch=gen_ch, name="myflattop2", ro_ch=ro_ch, 
                       style="flat_top", 
                       envelope="ramp", 
                       freq=cfg['freq'], 
                       length=0.1,
                       phase=90,
                       gain=1.0, 
                      )

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        self.pulse(ch=cfg['gen_ch'], name="myflattop", t=0)
        self.pulse(ch=cfg['gen_ch'], name="mygaus", t=0.4)
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0.8)
        self.pulse(ch=cfg['gen_ch'], name="myflattop2", t=1.2)
        self.pulse(ch=cfg['gen_ch'], name="mygaus", t=1.6)
        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': TRIG_TIME,
          'ro_len': 1.9,
         }

prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

if QICK_EMU is False:
    iq_list = prog.acquire_decimated(soc, rounds=1)
    t = prog.get_time_axis(ro_index=0)

    plt.plot(t, iq_list[0][:,0], label="I value")
    plt.plot(t, iq_list[0][:,1], label="Q value")
    plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
    plt.legend()
    plt.ylabel("a.u.")
    plt.xlabel("us")

print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)

macros:
	WriteReg(dst='s_core_w1', src=0)
	ConfigReadout(ch=0, name='myro', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3734ca0>}, t_regs={'t': 0})
	Delay(t=1.0, auto=True, gens=True, ros=True, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3734d90>}, t_regs={'t': 430})
	OpenLoop(n=1, name='reps')
	Trigger(ros=[0], tts=[], pins=[0], t=0.4, width=0.023251488095238096, ddr4=True, mr=False, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3734f10>, 'width': <qick.asm_v2.QickParam object at 0x7faca3734d60>}, t_regs={'t': 172, 'width': 10}, outdict=defaultdict(<class 'int'>, {}), trigset={0, 9, 10})
	Pulse(ch=0, name='myflattop', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3734dc0>}, t_regs={'t': 0})
	Pulse(ch=0, name='mygaus', t=0.4, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3734fd0>}, t_regs={'t': 172})
	Pulse(ch=0, name='myconst', t=0.8, tag=None, t_params={'t': <qick.asm_v2.QickP

In [11]:
# Compile the experiment
prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

# Dump the info required to run the experiment in the emulator
print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)

macros:
	WriteReg(dst='s_core_w1', src=0)
	ConfigReadout(ch=0, name='myro', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca36ac6a0>}, t_regs={'t': 0})
	Delay(t=1.0, auto=True, gens=True, ros=True, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca36ac610>}, t_regs={'t': 430})
	OpenLoop(n=1, name='reps')
	Trigger(ros=[0], tts=[], pins=[0], t=0.4, width=0.023251488095238096, ddr4=True, mr=False, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca36ace20>, 'width': <qick.asm_v2.QickParam object at 0x7faca36ac640>}, t_regs={'t': 172, 'width': 10}, outdict=defaultdict(<class 'int'>, {}), trigset={0, 9, 10})
	Pulse(ch=0, name='myflattop', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca36ac550>}, t_regs={'t': 0})
	Pulse(ch=0, name='mygaus', t=0.4, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca36ac3a0>}, t_regs={'t': 172})
	Pulse(ch=0, name='myconst', t=0.8, tag=None, t_params={'t': <qick.asm_v2.QickP

## many envelopes pulses program

Test of several pulses generation using different envelope types (gauss, cosine, triangular, drag)


In [12]:
GEN_CH = 0
RO_CH = 0
TRIG_TIME = 0.40
FREQ = 100

class ManyEnvelopesPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        gauss_len = 0.1
        self.add_gauss(ch=gen_ch, name="gauss", sigma=gauss_len/10, length=gauss_len, even_length=True)
        cosine_len = 0.15
        self.add_cosine(ch=gen_ch, name="cosine", length=cosine_len, even_length=True)
        triangle_len = 0.1
        self.add_triangle(ch=gen_ch, name="triangle", length=triangle_len, even_length=True)
        drag_len = 0.15
        self.add_DRAG(ch=gen_ch, name="drag", sigma=drag_len/10, delta=10, alpha=0.5, length=drag_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="mygauss", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="gauss", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mycosine", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="cosine", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mytriangle", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="triangle", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mydrag", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="drag", 
                       freq=cfg['freq'], 
                       phase=0,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mycosine90", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="cosine", 
                       freq=cfg['freq'], 
                       phase=90,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mycosine180", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="cosine", 
                       freq=cfg['freq'], 
                       phase=180,
                       gain=1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="mycosine270", ro_ch=ro_ch, 
                       style="arb", 
                       envelope="cosine", 
                       freq=cfg['freq'], 
                       phase=-90,
                       gain=1.0, 
                      )



        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        self.pulse(ch=cfg['gen_ch'], name="mygauss", t=0)
        self.pulse(ch=cfg['gen_ch'], name="mycosine", t=0.2)
        self.pulse(ch=cfg['gen_ch'], name="mytriangle", t=0.4)
        self.pulse(ch=cfg['gen_ch'], name="mydrag", t=0.8)
        self.pulse(ch=cfg['gen_ch'], name="mycosine90", t=1)
        self.pulse(ch=cfg['gen_ch'], name="mycosine180", t=1.2)
        self.pulse(ch=cfg['gen_ch'], name="mycosine270", t=1.4)

config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': TRIG_TIME,
          'ro_len': 2.0,
         }

prog = ManyEnvelopesPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

if QICK_EMU is False:
    iq_list = prog.acquire_decimated(soc, rounds=1)
    t = prog.get_time_axis(ro_index=0)

    plt.plot(t, iq_list[0][:,0], label="I value")
    plt.plot(t, iq_list[0][:,1], label="Q value")
    plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
    plt.legend()
    plt.ylabel("a.u.")
    plt.xlabel("us")

print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)

macros:
	WriteReg(dst='s_core_w1', src=0)
	ConfigReadout(ch=0, name='myro', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca351fe50>}, t_regs={'t': 0})
	Delay(t=1.0, auto=True, gens=True, ros=True, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca372d190>}, t_regs={'t': 430})
	OpenLoop(n=1, name='reps')
	Trigger(ros=[0], tts=[], pins=[0], t=0.4, width=0.023251488095238096, ddr4=True, mr=False, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca372d7f0>, 'width': <qick.asm_v2.QickParam object at 0x7faca372d1c0>}, t_regs={'t': 172, 'width': 10}, outdict=defaultdict(<class 'int'>, {}), trigset={0, 9, 10})
	Pulse(ch=0, name='mygauss', t=0, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca372d970>}, t_regs={'t': 0})
	Pulse(ch=0, name='mycosine', t=0.2, tag=None, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca372d760>}, t_regs={'t': 86})
	Pulse(ch=0, name='mytriangle', t=0.4, tag=None, t_params={'t': <qick.asm_v2.Qic

## tProc basic functionality
Write Data Registers

In [13]:
from qick.asm_v2 import QickProgramV2

prog = QickProgramV2(soccfg)
prog.declare_readout(ch=0, length=100)
prog.label('reg_wr_test')
# Write all the Data registers with some value
prog.write_reg(dst='r0',  src=12345678)
prog.write_reg(dst='r1',  src=12345678)
prog.write_reg(dst='r2',  src=12345678)
prog.write_reg(dst='r3',  src=12345678)
prog.write_reg(dst='r4',  src=12345678)
prog.write_reg(dst='r5',  src=12345678)
prog.write_reg(dst='r6',  src=12345678)
prog.write_reg(dst='r7',  src=12345678)
prog.wait(t=1)
prog.write_reg(dst='r8',  src=12345678)
prog.write_reg(dst='r9',  src=12345678)
prog.write_reg(dst='r10', src=12345678)
prog.write_reg(dst='r11', src=12345678)
prog.write_reg(dst='r12', src=12345678)
prog.write_reg(dst='r13', src=12345678)
prog.write_reg(dst='r14', src=12345678)
prog.write_reg(dst='r15', src=12345678)
# Write all the Data registers to 0
prog.wait(t=2)
prog.write_reg(dst='r0',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r1',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r2',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r3',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r4',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r5',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r6',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r7',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r8',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r9',  src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r10', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r11', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r12', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r13', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r14', src='s_zero')
prog.wait(t=0.1)
prog.write_reg(dst='r15', src='s_zero')

prog.label('time_test')
prog.wait(t=3)
# Test the Dispatcher execution time
for N in range(11,31):
    prog.write_reg(dst='r15', src=N)
    prog.trigger(ros=[0], pins=[0], t=prog.cycles2us(2**N+15))

# Increase usr_time to test upper time bits
# Need to force internal register because tProc can't write it directly
# use an internal register to sync the testbench
# Wait for time to be 2**30
prog.wait(t=prog.cycles2us(2**30))
# Increase reference to 2**30
prog.delay(t=prog.cycles2us(2**30))

for N in range(31,49):
    prog.write_reg(dst='r15', src=N)
    # Force reference from testbench to be 2**N
    # prog.delay(t=prog.cycles2us(2**N))
    # Wait 1us
    prog.wait(t=1)
    # Push trigger after another 1us
    prog.trigger(ros=[0], pins=[0], t=1)
prog.end()


print(prog)
prog.print_pmem2hex()


Readout time 9 appears to conflict with previous readout ending at 104.796782?
Readout time 19 appears to conflict with previous readout ending at 109.558687?
Readout time 38 appears to conflict with previous readout ending at 119.082496?
Readout time 76 appears to conflict with previous readout ending at 138.130115?
Readout time 152 appears to conflict with previous readout ending at 176.225353?
Readout time 1 appears to conflict with previous readout ending at 100.034877?
Readout time 1 appears to conflict with previous readout ending at 101.000000?
Readout time 1 appears to conflict with previous readout ending at 101.000000?
Readout time 1 appears to conflict with previous readout ending at 101.000000?
Readout time 1 appears to conflict with previous readout ending at 101.000000?
Readout time 1 appears to conflict with previous readout ending at 101.000000?
Readout time 1 appears to conflict with previous readout ending at 101.000000?
Readout time 1 appears to conflict with previou

macros:
	Label(label='reg_wr_test')
	WriteReg(dst='r0', src=12345678)
	WriteReg(dst='r1', src=12345678)
	WriteReg(dst='r2', src=12345678)
	WriteReg(dst='r3', src=12345678)
	WriteReg(dst='r4', src=12345678)
	WriteReg(dst='r5', src=12345678)
	WriteReg(dst='r6', src=12345678)
	WriteReg(dst='r7', src=12345678)
	Wait(t=1, auto=False, tag=None, no_warn=False, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3852550>}, t_regs={'t': 430})
	WriteReg(dst='r8', src=12345678)
	WriteReg(dst='r9', src=12345678)
	WriteReg(dst='r10', src=12345678)
	WriteReg(dst='r11', src=12345678)
	WriteReg(dst='r12', src=12345678)
	WriteReg(dst='r13', src=12345678)
	WriteReg(dst='r14', src=12345678)
	WriteReg(dst='r15', src=12345678)
	Wait(t=2, auto=False, tag=None, no_warn=False, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca3852c40>}, t_regs={'t': 860})
	WriteReg(dst='r0', src='s_zero')
	Wait(t=0.1, auto=False, tag=None, no_warn=False, t_params={'t': <qick.asm_v2.QickParam object at 0x7faca38520a0>}

## Test issue 359

In [14]:
from qick.asm_v2 import QickProgramV2, AsmInst

prog = QickProgramV2(soccfg)
prog.declare_readout(ch=0, length=100)
prog.label('arith_test')
prog.write_reg(dst='r0',  src=10)
prog.write_reg(dst='r1',  src=3)
# inst = AsmInst(inst={'CMD':"REG_WR", 'DST': 's2', 'SRC':'imm', 'LIT':'#49'}, addr_inc=1)
# prog.append_macro(inst)
# inst = AsmInst(inst={'CMD':"ARITH", 'C_OP': 'T', 'R1':'r0', 'R2':'r1'}, addr_inc=1)
# prog.append_macro(inst)
# inst = AsmInst(inst={'CMD': 'JUMP', 'IF': 'NF', 'ADDR': '&5'}, addr_inc=1)
# prog.append_macro(inst)
inst = AsmInst(inst={'CMD': 'PA', 'C_OP': '3', 'R1':'r0', 'R2':'r1'}, addr_inc=1)
prog.append_macro(inst)
prog.nop()
prog.nop()
prog.nop()
prog.nop()
prog.nop()
inst = AsmInst(inst={'CMD': 'PB', 'C_OP': '3', 'R1':'s6', 'R2':'r0', 'R3':'r1'}, addr_inc=1)
prog.append_macro(inst)
prog.nop()
prog.nop()
prog.nop()
prog.nop()
prog.nop()
prog.write_reg(dst='r2',  src=1234)
prog.write_reg(dst='r3',  src='s6')
prog.write_reg(dst='r4',  src='s7')

prog.end()

print(prog)

prog.print_pmem2hex()

macros:
	Label(label='arith_test')
	WriteReg(dst='r0', src=10)
	WriteReg(dst='r1', src=3)
	AsmInst(inst={'CMD': 'PA', 'C_OP': '3', 'R1': 'r0', 'R2': 'r1'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'PB', 'C_OP': '3', 'R1': 's6', 'R2': 'r0', 'R3': 'r1'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	AsmInst(inst={'CMD': 'NOP'}, addr_inc=1)
	WriteReg(dst='r2', src=1234)
	WriteReg(dst='r3', src='s6')
	WriteReg(dst='r4', src='s7')
	End()
registers:
pulses:
expanded ASM:
	arith_test:
	     NOP 
	arith_test:
	     REG_WR r0 imm #10 
	     REG_WR r1 imm #3 
	     PA 3 r0 r1 
	     NOP 
	     NOP 
	     NOP 
	     NOP 
	     NOP 
	     PB 3 s6 r0 r1 
	     NOP 
	

## Test issue 361

In [ ]:
class RampProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        self.declare_gen(ch=gen_ch, nqz=1)
            
        self.add_triangle(gen_ch, "myenv", cfg['pulse_len'])
        self.add_pulse(ch=gen_ch,
                      name="mypulse",
                      ro_ch=ro_ch,
                      style="arb",
                      freq=cfg['freq'],
                      envelope="myenv",
                      phase=cfg['phase'],
                      gain=cfg['gain'],
                      )

        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        self.add_readoutconfig(ch=ro_ch,
                              name="myro",
                              freq=cfg['freq'],
                              gen_ch=gen_ch)

        self.send_readoutconfig(ch=ro_ch,
                                   name="myro",
                                   t=0)

    def _body(self, cfg):
        self.delay(1.0)
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)
        
config = \
{
    'gen_ch': 0,
    'ro_ch': 0,
    'freq': 0,
    'trig_time': 0.44,
    'ro_len': 0.05,
    'pulse_len': 0.1,
    'gain': 1.0,
    'phase': 0.0
}

prog = RampProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

# Test the decimated buffer
iq_list = prog.acquire_decimated(soc)
plt.plot(iq_list[0][:,0], label='I')
plt.plot(iq_list[0][:,1], label='Q')
plt.xlabel('time [samples]')
plt.ylabel('amplitude [arb. units]')
plt.legend()

# print(iq_list[:10])

print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
# prog.print_sg_mem(sg_idx=0, gen_file=True)


In [ ]:
# Start Debug Bridge Server if present
try:
    bridge = soc._get_block('debug_bridge_0')
    bridge.start_xvc_server(bufferLen=4096, serverAddress="0.0.0.0", serverPort=2542, reconnect=True, verbose=True)

except:
    print('Debug blocks not found')

In [ ]:
## Do not forget to stop the XVC server once you finish.
bridge.stop_xvc_server()

# Qubit Emulator 
Frequency Sweep

In [ ]:
class FrequencySweepProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch, outsel='product')

         # self.add_gauss(ch=gen_ch, name="ramp", sigma=cfg['ramp_len']/10, length=cfg['ramp_len'], even_length=True)
        self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch, 
                       style="const", 
                    #    envelope="ramp", 
                       freq=cfg['freq'], 
                       length=cfg['flat_len'],
                       phase=cfg['phase'],
                       gain=cfg['gain'], 
                      )
        # self.add_triangle(ch=gen_ch, name="ramp", length=cfg['ramp_len'], even_length=True)
        # self.add_pulse(ch=gen_ch, name="mypulse", ro_ch=ro_ch, 
        #                style="arb", 
        #                envelope="ramp", 
        #                freq=cfg['freq'], 
        #                phase=cfg['phase'],
        #                gain=cfg['gain'], 
        #               )

        self.add_loop("freq_sweep", self.cfg["steps"])

    def _body(self, cfg):
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=cfg['ro_ch'], name="myro", t=0)
        self.pulse(ch=cfg['gen_ch'], name="mypulse", t=0.1)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])

In [ ]:
GEN_CH = 0
RO_CH = 0
FREQ = 100
DELTA_FREQ = 10

# do a sweep with STEPS points and plot decimated
config = {'steps': 11,
          'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': QickSweep1D("freq_sweep", FREQ-DELTA_FREQ, FREQ+DELTA_FREQ),
          'trig_time': 0.5,
          'ro_len': 1.0,
          'flat_len': 0.8,
          'ramp_len': 0.1,
          'phase': 0,
          'gain': 0.5
         }

# prog = FrequencySweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
# iq_list = prog.acquire_decimated(soc, rounds=1)

# plt.figure()
# t = prog.get_time_axis(ro_index=0)
# for ii, iq in enumerate(iq_list[0]):
#     # plt.plot(t, iq[:,0], label="I value, step %d"%(ii))
#     # plt.plot(t, iq[:,1], label="Q value, step %d"%(ii))
#     plt.plot(t, np.abs(iq.dot([1,1j])), label="mag, step %d"%(ii))
# plt.legend()
# plt.ylabel("a.u.")
# plt.xlabel("us")


prog = FrequencySweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)
freqs = prog.get_pulse_param('myro', 'freq', as_array=True)
print(freqs)
iq_list = prog.acquire(soc, rounds=1, progress=True)

plt.figure()
plt.plot(freqs, np.abs(iq_list[0][0].dot([1,1j])))
# plt.plot(np.angle(iq_list[0][0].dot([1,1j]), deg=True))
# plt.plot(iq_list[0][0,:,0], iq_list[0][0,:,1])
plt.ylabel("amp")
plt.xlabel("freq")


In [ ]:

print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)


# Test issue 53 - SG ready to CDCSYNC & tPROC bug test
https://github.com/openquantumhardware/qick_internal/issues/53

In [ ]:
GEN_CH = 0
RO_CH = 0
FREQ = 100
# FREQ = 0

class MultiPulseProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'])

        ramp_len = self.cycles2us(20, gen_ch=gen_ch)
        self.add_gauss(ch=gen_ch, name="ramp", sigma=ramp_len/10, length=ramp_len, even_length=True)
        
        self.add_pulse(ch=gen_ch, name="myflattop", ro_ch=ro_ch, 
                       style    = "flat_top", 
                       envelope = "ramp", 
                       freq     = cfg['freq'], 
                       length   = 0.1,
                       phase    = 0,
                       gain     = 1.0, 
                      )

        self.add_pulse(ch=gen_ch, name="myflattop2", ro_ch=ro_ch, 
                       style    = "flat_top", 
                       envelope = "ramp", 
                       freq     = cfg['freq'], 
                       length   = 0.1,
                       phase    = 90,
                       gain     = 0.5, 
                      )

        self.add_readoutconfig(ch=ro_ch, name="myro", freq=cfg['freq'], gen_ch=gen_ch)
        # send the config to the dynamic RO
        self.send_readoutconfig(ch=ro_ch, name="myro", t=0)
        
    def _body(self, cfg):
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'], ddr4=True)

        for i in range(0,15):
            self.pulse(ch=cfg['gen_ch'], name="myflattop", t=1.0)
            self.pulse(ch=cfg['gen_ch'], name="myflattop2", t=1.0)
        
config = {'gen_ch': GEN_CH,
          'ro_ch': RO_CH,
          'freq': FREQ,
          'trig_time': 1.0,
          'ro_len': 5.0,
         }

prog = MultiPulseProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

iq_list = prog.acquire_decimated(soc, rounds=10)
t = prog.get_time_axis(ro_index=0)

plt.figure(figsize=[20,10])
plt.plot(t, iq_list[0][:,0], label="I value")
plt.plot(t, iq_list[0][:,1], label="Q value")
plt.plot(t, np.abs(iq_list[0].dot([1,1j])), label="magnitude")
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us")

print(prog)
prog.print_pmem2hex()
prog.print_wmem2hex()
prog.print_sg_mem(sg_idx=0,gen_file=True)